## Intro to PyTorch

Torch is a numerical computation library like Numpy but instead of arrays, it uses tensors. 

Tensors behave the same way as arrays but PyTorch has extra functionality under the hood which builds a computation graph as you perform calculations on the tensors and calculate gradients with one line of code.

### Linear regression

In [42]:
def sample_linear_data(m=20):
    X = ##
    Y = ##
    return X, Y #returns X (the input) which should have shape (m, 2) and Y (labels) which should have shape (m, 1)

In [ ]:
X, Y = sample_linear_data()
print('X:',X, '\n')
print('Y:',Y, '\n')

#### From scratch

#### Using torch.nn.Module